In [1]:
from procDataSet import TrainingQuery
from bert_serving.client import BertClient
from multiprocessing import Pool
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim  as optim
import argparse
from keras.utils import to_categorical
import pandas as pd
import numpy as np
from torch.functional import F
import os

Using TensorFlow backend.


In [ ]:
bc = BertClient(ip='localhost', check_version=False, check_length=False)
class TrainingQuery(Dataset):
    def __init__(self, raw_training_data, news_urls, contents):
        # read provided data
        # proccess data
        merged_training = pd.merge(raw_training_data, news_urls, on=['News_Index'])

        # construct dataset
        self.queries = merged_training['Query']
        self.contents = contents[merged_training['News_URL']]
        self.targets = merged_training['Relevance']
        
        self.contents = list(self.contents)
        self.queries = list(self.queries)
        
#         self.contents = np.array([ bc.encode(pad_sentence(text.split('。'), 50)) for text in self.contents])
#         self.queries = bc.encode(self.queries)
        
        self.size = len(merged_training)
        
    def __getitem__(self, i):
        return self.queries[i], self.contents[i], self.targets[i]
    
    def __len__(self):
        return self.size


In [ ]:
class LSTM_Net(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, num_layers, dropout=0.5):
        super(LSTM_Net, self).__init__()
        # Fix/Train embedding 
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.dropout = dropout
        self.title_lstm = nn.LSTM(embedding_dim,
                            hidden_dim,
                            dropout=dropout,
                            num_layers=num_layers,
                            batch_first=True,
                            bidirectional=True)
        
        self.content_lstm = nn.LSTM(embedding_dim,
                            hidden_dim,
                            dropout=dropout,
                            num_layers=num_layers,
                            batch_first=True,
                            bidirectional=True)
        
#         self.regressor = nn.Sequential(
#                             nn.Dropout(dropout),
#                             nn.Linear(hidden_dim*2, 1),
#                             nn.SELU())

        self.regressor = nn.Sequential(
                            nn.Dropout(dropout),
                            nn.Linear(hidden_dim*2, 4),
                            nn.Softmax())

    def forward(self, titles, contents):
        titles = titles.view(titles.size(0), 1, titles.size(1))
#         titles = titles.repeat(1, 50, 1)
#         output = F.cosine_similarity(titles, contents,dim=1, eps=1e-8) * 2 + 2
        t, _ = self.title_lstm(titles, None)
        c, _ = self.content_lstm(contents, None)
        t = t[:, -1, :]
        c = c[:, -1, :]
        output = self.regressor(c*t)
#         output = F.cosine_similarity(t,c,dim=1, eps=1e-8) * 2 + 2
#         print(output)
#         print(output.shape)
        return output
    
def evaluation(outputs, labels):
    values, indices = torch.max(outputs, 1)
    correct = torch.mean(torch.tensor(labels==indices, dtype=torch.float))
#     correct = torch.mean((outputs-labels)**2)
    return correct

In [ ]:
def pad_sentence(text_token, max_len):
    text_token = [ i for i in text_token if len(i) > 1 and set(i) != set(['\n']) ]
    text_token_len = len(text_token)
    if max_len > text_token_len:
        text_token = text_token  +  ["。"] * ( max_len - text_token_len)
    else:
        text_token = text_token[:max_len]     
    return text_token

In [ ]:
def training(args, train, valid, model, device, model_name):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    model.train()
    batch_size, n_epoch = args.batch, args.epoch
#     criterion = nn.MSELoss()
    criterion = nn.CrossEntropyLoss()
    t_batch = len(train) 
    v_batch = len(valid) 

    optimizer = optim.Adam(model.parameters(), lr=args.lr)
    
    total_loss, total_acc, best_acc = 0, 0, 0
    train_acc_list = []
    val_acc_list = []
    
    for epoch in range(n_epoch):
        total_loss, total_acc = 0, 0      
        # training set
        for i, (titles, contents ,labels) in enumerate(train):      
            titles = titles.to(device)
            contents = contents.to(device)
            labels = labels.to(device)
# #             labels = labels.to(device, dtype=torch.float)
            
            optimizer.zero_grad()
            outputs = model(titles, contents)
            outputs = outputs.squeeze()
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            correct = evaluation(outputs, labels)
            total_acc += (correct / batch_size)
            total_loss += loss.item()
            
            print('[ Epoch{} == {}/{} ] loss:{:.3f} acc {:.3f}'.format(
            	epoch+1, i+1, t_batch, loss.item(), correct*100 / batch_size), end='\r')
            
        print('\nTrain | Loss:{:.5f} acc {:.3f}'.format(total_loss / t_batch, total_acc / t_batch*100))
        
        with torch.no_grad():
            total_loss, total_acc = 0, 0
            for i, (titles, contents ,labels) in enumerate(valid):
                titles = titles.to(device)
                contents = contents.to(device)
                labels = labels.to(device)
#                 labels = labels.to(device, dtype=torch.float)
                
                outputs = model(titles, contents)
                outputs = outputs.squeeze()
                loss = criterion(outputs, labels)
                correct = evaluation(outputs, labels)
                total_acc += (correct / batch_size)
                total_loss += loss.item()
            
            print("Valid | Loss:{:.5f} acc {:.3f}".format(total_loss/v_batch, total_acc/v_batch*100))
            if total_acc > best_acc:
                best_acc = total_acc
                if total_acc / v_batch * 100 > 75:
                    torch.save(model, "{}/mod_{}_ckpt_{:.3f} acc {:.3f}".format('bag_4/', model_name, total_acc/v_batch*100))
                    print('saving model with acc {:.3f}'.format(total_acc/v_batch*100))
            
            val_acc_list.append(total_acc / v_batch) 
            
    print(np.min(val_acc_list))

In [ ]:
class ARGS():
    lr = 0.0001
    batch = 50
    epoch = 100
    num_layers = 2
    seq_len = 30
    word_dim =  768
    hidden_dim = 200

In [10]:
from sklearn.model_selection import train_test_split
folder = 'news_data_1/'
raw_training_data = pd.read_csv(os.path.join(folder,'TD.csv'))
news_urls = pd.read_csv(os.path.join(folder,'NC_1.csv'))
contents = pd.read_json(os.path.join(folder,'url2content.json'), typ=pd.Series)
train_data, test_data = train_test_split(raw_training_data, test_size=0.1)
train_dataset = TrainingQuery(train_data, news_urls, contents)
test_dataset = TrainingQuery(test_data, news_urls, contents)

In [ ]:
import jieba
sentenece_arr = []
# for text in contents:
#     for sen in text.split('\n'):      
ans= [ sentenece_arr.append(jieba.lcut(sen)) for text in contents for sen in text.split('\n') ]

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
# step 1
vectoerizer = CountVectorizer(min_df=1, max_df=1.0, token_pattern='\\b\\w+\\b')
# step 2
vectoerizer.fit(sentenece_arr)
# step 3
bag_of_words = vectoerizer.get_feature_names()

In [29]:
X = vectoerizer.transform(sentenece_arr)

In [42]:
for cnt, i in enumerate(X):
    print(i)
    if cnt == 2:
        break

  (0, 295492)	1
  (0, 450110)	1
  (0, 761655)	1
  (0, 1323573)	1
  (0, 2291416)	1
  (0, 2585515)	1
  (0, 2714842)	1
  (0, 3059746)	1
  (0, 3754082)	1
  (0, 457540)	1
  (0, 1078858)	1
  (0, 1252197)	1
  (0, 3057070)	1
  (0, 3096302)	1
  (0, 3245816)	1
  (0, 739816)	1
  (0, 1482892)	1
  (0, 1872955)	1
  (0, 2556991)	1
  (0, 3059746)	1
  (0, 3212055)	1
  (0, 4757022)	1
  (0, 5061817)	1


In [37]:
from sklearn.feature_extraction.text import TfidfTransformer
# step 1
tfidf_transformer = TfidfTransformer()
# step 2
tfidf_transformer.fit(X.toarray())
# step 3
for idx, word in enumerate(vectoerizer.get_feature_names()):
    print("{}\t{}".format(word, tfidf_transformer.idf_[idx]))
    break
    
# step 4
tfidf = tfidf_transformer.transform(X)

MemoryError: 

In [7]:
#coding=utf-8
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_model = TfidfVectorizer().fit(sentenece_arr)

In [149]:
args = ARGS()

# Get model
train_data = DataLoader(train_dataset, batch_size=args.batch)
valid_data = DataLoader(test_dataset, batch_size=args.batch)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTM_Net(args.word_dim, args.hidden_dim, args.num_layers)
model = model.to(device)

training(args, train_data, valid_data, model, device,'test.h5')

/home/hanju/miniconda3/envs/ml/lib/python3.6/site-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/home/hanju/miniconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


[ Epoch1 == 86/86 ] loss:1.257 acc 0.941
Train | Loss:1.33812 acc 0.895
Valid | Loss:1.27451 acc 0.872
[ Epoch2 == 86/86 ] loss:1.216 acc 1.176
Train | Loss:1.24470 acc 0.926
Valid | Loss:1.23823 acc 0.984
[ Epoch3 == 86/86 ] loss:1.265 acc 0.706
Train | Loss:1.21738 acc 0.994
Valid | Loss:1.22502 acc 0.992
[ Epoch4 == 86/86 ] loss:1.232 acc 0.824
Train | Loss:1.20767 acc 1.017
Valid | Loss:1.22205 acc 0.992
[ Epoch5 == 86/86 ] loss:1.247 acc 0.824
Train | Loss:1.20435 acc 1.032
Valid | Loss:1.22032 acc 1.004
[ Epoch6 == 86/86 ] loss:1.265 acc 0.706
Train | Loss:1.20262 acc 1.033
Valid | Loss:1.22125 acc 1.020
[ Epoch7 == 86/86 ] loss:1.282 acc 0.824
Train | Loss:1.20184 acc 1.038
Valid | Loss:1.22143 acc 1.004
[ Epoch8 == 86/86 ] loss:1.291 acc 0.588
Train | Loss:1.20155 acc 1.033
Valid | Loss:1.21953 acc 1.008
[ Epoch9 == 86/86 ] loss:1.282 acc 0.824
Train | Loss:1.20082 acc 1.038
Valid | Loss:1.21502 acc 1.036
[ Epoch10 == 86/86 ] loss:1.278 acc 0.824
Train | Loss:1.20018 acc 1.036


KeyboardInterrupt: 

In [25]:
from sklearn.model_selection import train_test_split
batch_size = 100000
folder = 'news_data_1/'
test_query = list(pd.read_csv('./data/QS_1.csv').Query)
test_loader = DataLoader(test_query, batch_size=args.batch, shuffle=False)

In [151]:
class TestQuery(Dataset):
    def __init__(self, contents, test_query):      
        # read data
        self.contents = contents
        self.queries = bc.encode(test_query)
        self.indices = list(product(range(self.queries.shape[0]), range(self.contents.shape[0])))
        self.size = len(self.indices)
        
    def __getitem__(self, i):    
        j, k = self.indices[i]
        return self.queries[j], self.contents[k]
    
    def __len__(self):
        return self.size

In [152]:
from itertools import product
test_data = TestQuery(test_dataset.contents, test_query)

In [153]:
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

In [160]:
results = []
model.eval()

with torch.no_grad():
    for q, c in test_loader:
        q, c = q.cuda(), c.cuda()
        outputs = model(q, c)
        results.append(outputs.cpu().data.numpy())   
        
results = np.concatenate(results, axis=0)
# results = np.sum(results * np.array([1,2,3,4]), axis=1)

In [161]:
for i in range(4):
    num = np.count_nonzero(np.argmax(results,1) == i)
    print('relevance {}: {}'.format(i,num))

relevance 0: 0
relevance 1: 6650
relevance 2: 2850
relevance 3: 0


In [168]:
results = results.reshape(20, -1)
search_result = np.flip(np.argsort(results, axis=1), axis=1)

df = pd.DataFrame()
df['Query_Index'] = ['q_{:02d}'.format(i+1) for i in range(20)]

for i in range(300):
    df['Rank_{:03d}'.format(i+1)] = search_result[:, i]

for i, row in df.iterrows():
    df.iloc[i, 1:] = df.iloc[i, 1:].apply(lambda x: 'news_{:06d}'.format(x))
    
fname = 'simple5.csv'
df.to_csv('output/' + fname, index=False)

In [ ]:
import pickle 
with  open('test.pickle') as file:
    pickle.dump(, file)